In [29]:
import torch as torch
import syft as sy
hook = sy.TorchHook(torch)
import numpy as np

import torch
import torchvision
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms

In [6]:
jake = sy.VirtualWorker(hook, id="jake")
print("Jake has: " + str(jake._objects))

john = sy.VirtualWorker(hook, id="john")

Jake has: {52973500771: <Plan Plan id:52973500771 owner:jake Tags: #fss_eq_plan_1 built>
, 99696909270: <Plan Plan id:99696909270 owner:jake Tags: #fss_eq_plan_2 built>
, 80807883839: <Plan Plan id:80807883839 owner:jake Tags: #fss_comp_plan_1 built>
, 911630716: <Plan Plan id:911630716 owner:jake Tags: #fss_comp_plan_2 built>
, 90045197714: <Plan Plan id:90045197714 owner:jake Tags: #xor_add_1 built>
, 51137915516: <Plan Plan id:51137915516 owner:jake Tags: #xor_add_2 built>
, 81187714074: <Plan Plan id:81187714074 owner:jake Tags: #fss_eq_plan_1 built>
, 88382600198: <Plan Plan id:88382600198 owner:jake Tags: #fss_eq_plan_2 built>
, 72296214479: <Plan Plan id:72296214479 owner:jake Tags: #fss_comp_plan_1 built>
, 59728129458: <Plan Plan id:59728129458 owner:jake Tags: #fss_comp_plan_2 built>
, 97167328842: <Plan Plan id:97167328842 owner:jake Tags: #xor_add_1 built>
, 93791304854: <Plan Plan id:93791304854 owner:jake Tags: #xor_add_2 built>
}


In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, )),
])

train_set = datasets.MNIST(
    "~/.pytorch/MNIST_data/", train=True, download=True, transform=transform)
test_set = datasets.MNIST(
    "~/.pytorch/MNIST_data/", train=False, download=True, transform=transform)



In [ ]:
federated_train_loader = sy.FederatedDataLoader(
    train_set.federate((jake, john)), batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=64, shuffle=True)

In [26]:
train_set.targets

tensor([5, 0, 4,  ..., 5, 6, 8])

In [27]:
train_set.data

tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0,

In [12]:
train_set

Dataset MNIST
    Number of datapoints: 60000
    Root location: /root/.pytorch/MNIST_data/
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )

In [8]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


model = Model()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [9]:
for epoch in range(0, 5):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader):
        # send the model to the client device where the data is present
        model.send(data.location)
        # training the model
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        # get back the improved model
        model.get()
        if batch_idx % 100 == 0: # print results each 100 iterations
            # get back the loss
            loss = loss.get()
            print('Epoch: {:2d} [{:5d}/{:5d} ({:3.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1,
                batch_idx * 64,
                len(federated_train_loader) * 64,
                100. * batch_idx / len(federated_train_loader),
                loss.item()))

Epoch:  1 [    0/60032 (  0%)]	Loss: 2.353447
Epoch:  1 [ 6400/60032 ( 11%)]	Loss: 1.348159
Epoch:  1 [12800/60032 ( 21%)]	Loss: 0.839848
Epoch:  1 [19200/60032 ( 32%)]	Loss: 0.701301
Epoch:  1 [25600/60032 ( 43%)]	Loss: 0.544994
Epoch:  1 [32000/60032 ( 53%)]	Loss: 0.552950
Epoch:  1 [38400/60032 ( 64%)]	Loss: 0.586103
Epoch:  1 [44800/60032 ( 75%)]	Loss: 0.486136
Epoch:  1 [51200/60032 ( 85%)]	Loss: 0.324459
Epoch:  1 [57600/60032 ( 96%)]	Loss: 0.410363
Epoch:  2 [    0/60032 (  0%)]	Loss: 0.478832
Epoch:  2 [ 6400/60032 ( 11%)]	Loss: 0.545559
Epoch:  2 [12800/60032 ( 21%)]	Loss: 0.277177
Epoch:  2 [19200/60032 ( 32%)]	Loss: 0.213412
Epoch:  2 [25600/60032 ( 43%)]	Loss: 0.242448
Epoch:  2 [32000/60032 ( 53%)]	Loss: 0.441177
Epoch:  2 [38400/60032 ( 64%)]	Loss: 0.506062
Epoch:  2 [44800/60032 ( 75%)]	Loss: 0.258631
Epoch:  2 [51200/60032 ( 85%)]	Loss: 0.303522
Epoch:  2 [57600/60032 ( 96%)]	Loss: 0.466407
Epoch:  3 [    0/60032 (  0%)]	Loss: 0.329046
Epoch:  3 [ 6400/60032 ( 11%)]	Los

In [10]:
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        test_loss += F.nll_loss(
            output, target, reduction='sum').item()
        # get the index of the max log-probability
        pred = output.argmax(1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss,
    correct,
    len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))


Test set: Average loss: 0.2492, Accuracy: 9281/10000 (93%)

